In [22]:

var fs = require("fs")
var tf = require("../../jslib/textFile")
var funcTools = require("../../jslib/funcTools")
var palette = require("../../jslib/palette.js")

var ls = directoryPath => (filterFileFunc = path => file => fs.statSync(path+file).isFile()) => {
    const files = fs.readdirSync(directoryPath,(err)=>{
        if (err) throw err;   
    })
    return {
        directory : directoryPath,
        files : files.filter(filterFileFunc(directoryPath))
    }
} 

var Plotly = require("ijavascript-plotly");

In [23]:
var resultPath = "../result/test/"
var dataPath = "../model/"

In [24]:


var metaFiles = ls(resultPath)(path => file =>  fs.statSync(path+file).isFile() && /^meta.*\.json$/.test(file))
var sampleFiles = ls(resultPath)(path => file =>  fs.statSync(path+file).isFile() && /.*\.csv$/.test(file))
var summaryFiles = ls(resultPath)(path => file => file === "summary.csv");
console.log(metaFiles)
console.log(sampleFiles)

var meta = JSON.parse(fs.readFileSync(`${metaFiles.directory}${metaFiles.files[0]}`))
console.log(meta)

var data = tf.text2Dataframe(fs.readFileSync(`${dataPath}${meta.data_file}`,"utf-8"),"csv")
var error = tf.text2Dataframe(fs.readFileSync(`${dataPath}${meta.data_file}`,"utf-8"),"csv")
var option = JSON.parse(fs.readFileSync("../model/"+meta.option_file,"utf-8"))


console.log(option)

var df_summarized = tf.text2Dataframe(fs.readFileSync(`${summaryFiles.directory}${summaryFiles.files[0]}`,"utf-8"),"csv")
console.log(Object.keys(df_summarized))

var Model = require("../"+meta.model)

var {
      model,
      parameters,
      updateCondition,
      constrain,
      magma
    } = Model(option)

var cmap = palette("tol-rainbow", parameters.length)

{ directory: '../result/test/',
  files: [ 'meta-2018_0713_024018.json' ] }
{ directory: '../result/test/',
  files:
   [ 'sample-0-2018_0713_024018.csv',
     'sample-1-2018_0713_024018.csv',
     'sample-10-2018_0713_024018.csv',
     'sample-11-2018_0713_024018.csv',
     'sample-12-2018_0713_024018.csv',
     'sample-13-2018_0713_024018.csv',
     'sample-14-2018_0713_024018.csv',
     'sample-15-2018_0713_024018.csv',
     'sample-2-2018_0713_024018.csv',
     'sample-3-2018_0713_024018.csv',
     'sample-4-2018_0713_024018.csv',
     'sample-5-2018_0713_024018.csv',
     'sample-6-2018_0713_024018.csv',
     'sample-7-2018_0713_024018.csv',
     'sample-8-2018_0713_024018.csv',
     'sample-9-2018_0713_024018.csv',
     'summary.csv' ] }
{ totalIteration: 200,
  workerNum: 4,
  alpha: 0.01,
  data_file: '1002-9-1C-opx1.csv',
  error_file: 'err_1002-9-1C-opx1_5per.csv',
  model: 'model/model-magma-next.js',
  acceptedTime:
   [ [ [Object],
       [Object],
       [Object],
       

In [40]:
var summarizedParameters = [...parameters];
var splited = df_summarized.parameter.map((k,i)=>{
    let parameter = k.slice(0,k.length-1)
    let num = k.slice(k.length-1)
    return [parameter,num]
})

splited.map(([parameter,num],i)=>{
    if (0 < i && i < splited.length -1){
        summarizedParameters[num][parameter] = df_summarized.mean[i]
    }
})

summarizedParameters[0]

{ MgN_beforeCrystallization: '91.92126584973485',
  growth_stoichiometry_orthopyroxene: '0.39652241931918203',
  growth_stoichiometry_spinel: '0.04850129325317029',
  MgN_beforeMixing: '91.2229393577133',
  mixing_stoichiometry_orthopyroxene: '0.24692972047844605',
  mixing_stoichiometry_spinel: '0.020197880718056456',
  log10_tau: '5.773611998142433' }

In [47]:
var parametersEachStage = {}
summarizedParameters.map((p,i)=>{
    if (i === 0){
        Object.keys(p).map(k=>{
          parametersEachStage[k] = []  
        })
    }
    Object.entries(p).map(([k,v])=>{
        parametersEachStage[k][i] = v
    })
})


[ undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined ]

In [48]:
Plotly([
    {
        y : parametersEachStage.growth_stoichiometry_orthopyroxene
    },
    {
        y : parametersEachStage.growth_stoichiometry_spinel
    }
])

In [49]:
Plotly([
    {
        y : parametersEachStage.log10_tau
    }
])

In [54]:
var modeled = model(summarizedParameters,data)
var ndProfiles = magma.diffusionProfiles.orthopyroxene;

var ndFe_Mg = ndProfiles.Fe_Mg.notDiffusedProfile.get()

Plotly([
    {
        x : modeled.x,
        y : modeled.Fe_Mg.map(v=>100/(1+v)),
        name : "Model: diffused"
    },
     {
        x : ndFe_Mg.x,
        y : ndFe_Mg.c.map(v=>100/(1+v)),
        name : "Model: original"
    },
    {
        x : data.x,
        y : data.Fe_Mg.map(v=>100/(1+parseFloat(v))),
        mode : "markers",
        name : "Observed"
    }

],{
    width : 800,
    height : 600,
    xaxis : {
        title : "Radius"
    },
    yaxis : {
        title : "Mg#"
    }
})

In [55]:
var ndCr2O3 = ndProfiles.Cr2O3.notDiffusedProfile.get()

Plotly([
    {
        x : modeled.x,
        y : modeled.Cr2O3,
        name : "Model: diffused"
    },
    {
        x : ndCr2O3.x,
        y : ndCr2O3.c,
        name : "Model: original"
    },
    {
        x : data.x,
        y : data.Cr2O3,
        mode : "markers",
        name : "Observed"
    }

],{
    width : 800,
    height : 600,
    xaxis : {
        title : "Radius"
    },
    yaxis : {
        title : "Cr2O3"
    }
})

In [36]:
var integratedLiquidLine = _ => {
    const  differentiationLines = magma.custom.differentiationLineStack;
    const  mixingLines = magma.custom.mixingLineStack;
    


    return (x,y) => funcTools.zip(mixingLines)(differentiationLines).map(
            ([m,d],i) => {
                if (i === 0){
                    return [
                    {
                        x : [m[x][0]],
                        y : [m[y][0]],
                        name : "Current WR",
                        mode : "markers",
                        marker : {
                            size : 8,
                            color : "#000000"
                        }
                    },
                    {
                        x : d[x],
                        y : d[y],
                        mode : "lines",
                        line : {
                            width : 2,
                            color : cmap[i]
                        }
                    },
                    {
                            x : m[x],
                            y : m[y],
                            mode : "markers",
                            marker : {
                                size : 4,
                                color : cmap[i]
                            }
                    }
                    ]
                }else{
                    return [
                        {
                            x : m[x],
                            y : m[y],
                            mode : "markers",
                            marker : {
                                size : 4,
                                color : cmap[i]
                            }
                        },
                        {
                            x : d[x],
                            y : d[y],
                            mode : "lines",
                            line : {
                                width : 2,
                                color : cmap[i]
                            }
                        }
                    ]
                }
            }
        ).reduce((a,b)=>[...a,...b],[])
        
}

var liquidLine = integratedLiquidLine()

//console.log(liquidLine("SiO2","MgO"))


In [37]:

Plotly(liquidLine("SiO2","MgO"),{
    width : 1000,
    height : 1000,
    xaxis : {
        title :"SiO2"
    },
    yaxis : {
        title : "MgO"
    }
})


In [35]:
Plotly(liquidLine("SiO2","Cr2O3"),{
    width : 1000,
    height : 1000,
    xaxis : {
        title :"SiO2"
    },
    yaxis : {
        title : "Cr2O3"
    }
})
